UNTESTED 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import dataset
Housing prices

In [ ]:
!kaggle competitions download -c house-prices-advanced-regression-techniques
!unzip house-prices-advanced-regression-techniques.zip
df = pd.read_csv('train.csv')

## Exploratory data analysis (EDA)

In [ ]:
# Display first few rows and basic information about the dataset
df.head()
df.info()

In [ ]:
# Summary statistics
print(df.describe())

In [ ]:
# Plot relationship between chosen features and target variable
plt.figure(figsize=(10, 6))
sns.scatterplot(x='GrLivArea', y='SalePrice', data=df)
plt.title('Relationship between GrLivArea (Above Grade Living Area) and Sale Price')
plt.xlabel('GrLivArea')
plt.ylabel('Sale Price')
plt.show()

Or, plot relationship between all features and target variable, by computing the correlations between variables

To do this, and to use all the features in the dataset, we need to make sure that all the features are numerical. We can do this by converting all the categorical features to numerical using one-hot encoding. We can then drop the original categorical features. We can also drop the Id column as it is not useful for our analysis.

In [ ]:
# Identify non-numeric columns
non_numeric_columns = df.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_columns)

In [ ]:
# Convert non-numeric columns using one-hot encoding
df_encoded = pd.get_dummies(df, columns=non_numeric_columns, drop_first=True)

In [ ]:
# Drop the Id column
df_encoded.drop('Id', axis=1, inplace=True)

In [2]:
# Get correlation between variables with respect to SalePrice
correlation = df_encoded.corr()
top_corr = correlation['SalePrice'].sort_values(ascending=False)[:20]
top_corr

NameError: name 'df_encoded' is not defined

In [ ]:
# Plot heatmap of correlation matrix for top 10 correlated features
plt.figure(figsize=(12, 10))
sns.heatmap(correlation.loc[top_corr.index, top_corr.index], annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap of Top 10 Features')
plt.show()

With the correlation matrix, you can see which features are correlated with the SalePrice, and which features are correlated with each other to identify dependent and independent variables. 

## Preprocessing

In [ ]:
# Check for missing values
missing = df.isnull().sum()
print(missing)

In [ ]:
# Check for duplicate rows
duplicates = df.duplicated().sum()

In [ ]:
# Drop columns with more than 50% missing values
df_cleaned = df.dropna(thresh=0.5*len(df), axis=1)
df_cleaned

In [ ]:
# Select features and target variable, e.g. 
X = df_cleaned[['GrLivArea', 'YearBuilt']]
y = df_cleaned['SalePrice']

In [ ]:
# Encode categorical variables (if not already done)
X = pd.get_dummies(X)

In [ ]:
# Split data into test and training sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Feature scaling 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Regression model training

In [ ]:
# Import regression models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42)
}

In [ ]:
# Train and evaluate models
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Model: {name}")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R^2 Score: {r2:.2f}")
    print("-------------------")

## Model evaluation and selection

In [ ]:
# Compare models visually
plt.figure(figsize=(10, 6))
for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    plt.scatter(y_test, y_pred, label=name)
plt.xlabel('Actual Sale Price')
plt.ylabel('Predicted Sale Price')
plt.title('Actual vs Predicted Sale Prices')
plt.legend()
plt.show()

# Discuss model performance and selection criteria (e.g., MSE, R^2)

## Advanced: optimise model hyperparameters

In [ ]:
# Run the models again with hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Hyperparameters for Decision Tree
param_grid_dt = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid Search for Decision Tree
grid_search_dt = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid_dt, cv=5)
grid_search_dt.fit(X_train_scaled, y_train)
best_dt = grid_search_dt.best_estimator_
y_pred_dt = best_dt.predict(X_test_scaled)
mse_dt = mean_squared_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)
print("Decision Tree:")
print(f"Best Parameters: {grid_search_dt.best_params_}")
print(f"Mean Squared Error: {mse_dt:.2f}")
print(f"R^2 Score: {r2_dt:.2f}")

In [ ]:
# Hyperparameters for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid Search for Random Forest
grid_search_rf = GridSearchCV(RandomForestRegressor(random_state=42), param_grid_rf, cv=5)
grid_search_rf.fit(X_train_scaled, y_train)
best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test_scaled)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print("Random Forest:")
print(f"Best Parameters: {grid_search_rf.best_params_}")
print(f"Mean Squared Error: {mse_rf:.2f}")
print(f"R^2 Score: {r2_rf:.2f}")
